# Haskell Language Primer

- [Programming Languages and Concepts](https://tramberend.beuth-hochschule.de/course/plc/)
- Summer 2019
- [Prof. Dr. Henrik Tramberend](https://tramberend.beuth-hochschule.de/)

# This is a JupyterLab Notebook

- Experiment for the interactive presentation of code
- It runs on an IHaskell kernel
- This is like running `ghci` on the command line

![Picture](ghci.png)


# Functions

Working with a simple poem

    Stay the patient course.
    Of little worth is your ire.
    The network is down


Put the poem into a string and bind that to a name.

In [1]:
poem = "Stay the patient course\nOf little worth is your ire\nThe network is down\n"

Apply some funtions to the poem. Output the poem to `stdout`.

In [2]:
putStrLn poem

Stay the patient course
Of little worth is your ire
The network is down

Function application is syntactically very reduced. The first expression is the function, the remaining expressions are the arguments.

Operators are just functions of two arguments in *infix notation*. Parantheses are just for grouping. Try to add a few numbers.

In [3]:
1 + 2 + 3 + 4

10

Extract a list of line strings from the text block.

In [4]:
lines poem

["Stay the patient course","Of little worth is your ire","The network is down"]

Extend the code above to reverse the order of the lines and reassemble the text block.

Haskell uses modules to structure programs. Functions and datatypes can be imported from other modules. Useful list functions are in the `Data.List` module. 

In [5]:
import Data.List

putStrLn (unlines (reverse (lines poem)))

The network is down
Of little worth is your ire
Stay the patient course

Define a function that does that. Function definition is also very reduced.

In [6]:
process text = unlines (reverse (lines text))

putStrLn (process poem)

The network is down
Of little worth is your ire
Stay the patient course

The function `process` can be written in a different style. The operator `.` composes functions, just like in function algebra:

\begin{equation*}
f(g(x)) = (f\cdot g)(x)
\end{equation*}

In [7]:
process text = (unlines . reverse . lines) text

Line 1: Eta reduce
Found:
process text = (unlines . reverse . lines) text
Why not:
process = (unlines . reverse . lines)

This is similar to the thread-last macro from Clojure

```clojure
(->> text 
     (lines)
     (reverse)
     (unlines))
```

`Eta reduction` removes unnecessary parameters from the function definition.

In [8]:
process = unlines . reverse . lines

putStrLn $ process poem

The network is down
Of little worth is your ire
Stay the patient course

The operator `$` helps to save parantheses. Haskellers don`t like parantheses. The following to function invokations are all equivalent:

```haskell
putStrLn (process poem)
putStrLn $ process poem
(putStrLn . process) poem
```

Define more functions like that on blocks of text.

In [9]:
reversLines   = unlines . reverse . lines
firstTwoLines = unlines . (take 2) . lines
lastLine      = unlines . (take 1 . reverse) . lines

In [10]:
putStrLn $ firstTwoLines poem

Stay the patient course
Of little worth is your ire

Line splitting and joining can be factored into a new function. This is *higher order programming* in Haskell.

In [11]:
byLine f = unlines . f . lines

reverseLines'  = byLine reverse
firstTwoLines' = byLine (take 2)
lastLine'      = byLine (take 1 . reverse)

In [12]:
putStrLn $ lastLine' poem

The network is down

Write a new function to indent the lines.

In [13]:
indent :: String -> String
indent s = "    " ++ s

The first line is the *type signature* of the function. Type signatures are basically comments for the programmer, the compile does not need them most of the time. It uses *type inference* to calculate types for lall and any expression the code.

`ghci` can be asked to print the type for any expression. Check types for `poem`, `indent` and `reverse`.

In [14]:
:type 

: 

Try to use the `indent` function.

In [15]:
indentLines = byLine indent

: 

What is the problem? Look at the expected and actual types. Which type does `byline` have? whicht type does `indent` have?

In [16]:
:type indent

indent :: String -> String

How to apply a function on strings to a list of strings?

Use `map` to do that. Which type does `map` have?

In [17]:
:type map

map :: forall a b. (a -> b) -> [a] -> [b]

`a` and `b` are *type variables* that state that map can be used on values of any type and lists on those values.

Use `map` to apply the `indent` function to a list of strings.

In [18]:
indentLines = byLine (map indent)

In [19]:
putStrLn $ indentLines poem

    Stay the patient course
    Of little worth is your ire
    The network is down

# Data 

Lists can be written in brackets `[]`. Lists can be constructed with the `:` operator which appends an element in front of a list.

Strings are just a list of characters.

In [20]:
'a':'b':[]

Line 1: Use list literal
Found:
'a' : 'b' : []
Why not:
['a', 'b']

"ab"

Functions can use *pattern matching* to perform different operations for different cases. Patterns are made of *constructors*.

Define function `count` to count the number of elements in a list.

In [21]:
count [] = 0
count (f:r) = 1 + count r

Line 1: Use foldr
Found:
count [] = 0
count (f : r) = 1 + count r
Why not:
count r = foldr (\ f -> (+) 1) 0 r

In [22]:
count "Hallo"
count [1,2,3,4]

5

4

There was no type signature defined for `count` and it could be applied to lists of characters and lists of numbers. What is type of `count`?

In [23]:
:type count

count :: forall p a. Num p => [a] -> p

`count` is automatically given the most generic type signature the compiler can find.

A *Tuple* is a fixed numbered collection of possibly heterogenous values. Define a tuple consisting of a number and a string.

In [24]:
(42, "hello")

(42,"hello")

Define a 3D vector type from tupel and define an add function. Use pattern matching.

In [25]:
type Vec3 = (Float, Float, Float)

add :: Vec3 -> Vec3 -> Vec3
add (x1, y1, z1) (x2, y2, z2) = (x1+x2, y1+y2, z1+z2)

In [26]:
:type add

add :: Vec3 -> Vec3 -> Vec3

*Algebraic data types* are composite user-defined types. Define a recursive binary tree type.

In [27]:
data Tree a = Empty
            | Leaf a
            | Node (Tree a) (Tree a)

Define a function that calculates the depth of a tree.

In [28]:
depth :: Tree a -> Int
depth Empty = 0
depth (Leaf _) = 1
depth (Node l r) = 1 + max (depth l) (depth r)

In [29]:
t = Node 
      (Node 
         (Leaf "a") 
         (Node 
            Empty 
            (Leaf "b")))
    Empty
depth t

4

Final and most important question? 

- Is this better than a text editor an GHCi in a shell?